<a href="https://colab.research.google.com/github/bhuguvi26/Copy-of-A-Comprehensive-ETL-Workflow-with-Python-for-Data-Engineers/blob/main/Copy_of_A_Comprehensive_ETL_Workflow_with_Python_for_Data_Engineers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import json
import requests
import xml.etree.ElementTree as ET
import sqlite3
import os
import sys

# -------------------------------------------------------
# CONFIG
# -------------------------------------------------------
BASE_URL = "https://raw.githubusercontent.com/bhuguvi26/Copy-of-A-Comprehensive-ETL-Workflow-with-Python-for-Data-Engineers/main/source%20(3)/"

FILES = [
    "source1.csv", "source2.csv", "source3.csv",
    "source1.json", "source2.json", "source3.json",
    "source1.xml", "source2.xml", "source3.xml"
]

DOWNLOAD_DIR = "downloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# -------------------------------------------------------
# DOWNLOAD FILES
# -------------------------------------------------------
def download_files():
    downloaded = []
    print("⬇️ Downloading files...\n")

    for file in FILES:
        url = BASE_URL + file
        save = os.path.join(DOWNLOAD_DIR, file)

        try:
            r = requests.get(url)
            if r.status_code == 200:
                with open(save, "wb") as f:
                    f.write(r.content)
                print(f"✅ Downloaded: {file}")
                downloaded.append(file)
            else:
                print(f"❌ Failed ({r.status_code}): {file}")
        except Exception as e:
            print(f"❌ Error downloading {file}: {e}")

    if not downloaded:
        sys.exit("❌ No files downloaded — check filenames!")

    print("\n📁 Download complete.\n")
    return downloaded

# -------------------------------------------------------
# JSON HANDLER (FIX FOR YOUR FILES)
# -------------------------------------------------------
def extract_json(path):
    rows = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rows.append(json.loads(line))
            except:
                pass
    return pd.DataFrame(rows)

# -------------------------------------------------------
# XML HANDLER
# -------------------------------------------------------
def extract_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()
    rows = []

    for rec in root.findall("./record"):
        row = {child.tag: child.text for child in rec}
        rows.append(row)

    return pd.DataFrame(rows)

# -------------------------------------------------------
# MAIN EXTRACT FUNCTION
# -------------------------------------------------------
def extract_all(downloaded):
    dfs = []

    for file in downloaded:
        path = os.path.join(DOWNLOAD_DIR, file)

        if file.endswith(".csv"):
            dfs.append(pd.read_csv(path))

        elif file.endswith(".json"):
            dfs.append(extract_json(path))

        elif file.endswith(".xml"):
            dfs.append(extract_xml(path))

    if not dfs:
        raise ValueError("❌ No data extracted")

    print("📤 All files extracted successfully.\n")
    return pd.concat(dfs, ignore_index=True)

# -------------------------------------------------------
# TRANSFORM
# -------------------------------------------------------
def transform(df):
    print("🔧 Transforming...\n")
    df.columns = df.columns.str.lower().str.strip()
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="ignore")
    return df

# -------------------------------------------------------
# LOAD → SQLITE
# -------------------------------------------------------
def load(df):
    print("💾 Loading into SQLite...\n")
    conn = sqlite3.connect("etl_output.db")
    df.to_sql("people", conn, if_exists="replace", index=False)
    conn.close()
    print("🎉 Load complete — etl_output.db created.\n")

# -------------------------------------------------------
# RUN ETL
# -------------------------------------------------------
downloaded = download_files()
df = extract_all(downloaded)
df = transform(df)
load(df)

print("🚀 ETL Pipeline Completed Successfully!")


⬇️ Downloading files...

✅ Downloaded: source1.csv
✅ Downloaded: source2.csv
✅ Downloaded: source3.csv
✅ Downloaded: source1.json
✅ Downloaded: source2.json
✅ Downloaded: source3.json
✅ Downloaded: source1.xml
✅ Downloaded: source2.xml
✅ Downloaded: source3.xml

📁 Download complete.

📤 All files extracted successfully.

🔧 Transforming...

💾 Loading into SQLite...

🎉 Load complete — etl_output.db created.

🚀 ETL Pipeline Completed Successfully!


/tmp/ipython-input-1234657921.py:113: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


# ReadMe
📊 ETL Pipeline in Python — CSV, JSON, XML | Google Colab

This project demonstrates a complete Extract, Transform, Load (ETL) workflow using Python in Google Colab.
The pipeline extracts data from CSV, JSON, and XML formats, transforms height and weight units, and loads the cleaned data into a CSV file for analytics or database storage.

🚀 Project Overview
✅ Objective

Build a production-style ETL pipeline that:

Extracts data from multiple formats (CSV, JSON, XML)

Transforms:

Height → meters

Weight → kilograms

Logs all ETL steps

Saves final clean dataset into transformed_data.csv

✅ Skills Used

Python

Pandas

File handling (CSV, JSON, XML)

Data transformation

Logging for ETL tracking

📁 Input Data Formats
CSV Example
name,height,weight
alex,65.78,112.99
ajay,71.52,136.49
alice,69.4,153.03

JSON Example
{"name":"jack","height":68.70,"weight":123.30}
{"name":"tom","height":69.80,"weight":141.49}

XML Example
<data>
   <person>
      <name>simon</name>
      <height>67.90</height>
      <weight>112.37</weight>
   </person>
</data>

📦 Output
transformed_data.csv preview:
name	height	weight	height_m	weight_kg
alex	65.78	112.99	1.671	51.251
ajay	71.52	136.49	1.817	61.911
alice	69.40	153.03	1.763	69.413
Generated Log File

etl_log.txt — contains timestamped logs for each ETL phase.

⚙️ ETL Workflow
1️⃣ Extract

Reads all uploaded .csv, .json, .xml files and combines into a DataFrame.

2️⃣ Transform

Height (inches → meters):
height_m = height * 0.0254

Weight (lbs → kg):
weight_kg = weight * 0.45359237

3️⃣ Load

Saves result to:

transformed_data.csv

📎 Running the Project in Google Colab
Step 1 — Upload Files
from google.colab import files
uploaded = files.upload()

Step 2 — Run Complete ETL Script

Run the one-cell ETL code provided in this project.

🧠 Key Learnings

ETL automation in Python

Parsing structured data files

Real-world logging practices

Data cleaning & unit conversion

✅ Project Status

✔ Completed
✔ Tested with real data
✔ Production-style logging & modularity